### Metrics
- for each dataset, load x and z, then compute silhouette score and trustworthiness
- save metrics to a pandas dataframe

In [8]:
import numpy as np
import pandas as pd

In [9]:
from sklearn.manifold import trustworthiness
from sklearn.metrics import silhouette_score

In [10]:
from tqdm.autonotebook import tqdm

In [11]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

In [12]:
output_dir = MODEL_DIR/'projections' 

In [13]:
#from https://gist.github.com/AlexandreAbraham/5544803
from sklearn.utils import check_random_state
from sklearn.metrics.pairwise import distance_metrics
from sklearn.metrics.pairwise import pairwise_distances
from joblib import Parallel, delayed
from itertools import combinations


def silhouette_score_block(X, labels, metric='euclidean', sample_size=None,
                           random_state=None, n_jobs=1, **kwds):
    """Compute the mean Silhouette Coefficient of all samples.
    The Silhouette Coefficient is calculated using the mean intra-cluster
    distance (a) and the mean nearest-cluster distance (b) for each sample.
    The Silhouette Coefficient for a sample is ``(b - a) / max(a, b)``.
    To clarrify, b is the distance between a sample and the nearest cluster
    that b is not a part of.
    This function returns the mean Silhoeutte Coefficient over all samples.
    To obtain the values for each sample, use silhouette_samples
    The best value is 1 and the worst value is -1. Values near 0 indicate
    overlapping clusters. Negative values generally indicate that a sample has
    been assigned to the wrong cluster, as a different cluster is more similar.
    Parameters
    ----------
    X : array [n_samples_a, n_features]
        Feature array.
    labels : array, shape = [n_samples]
             label values for each sample
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string, it must be one of the options
        allowed by metrics.pairwise.pairwise_distances. If X is the distance
        array itself, use "precomputed" as the metric.
    sample_size : int or None
        The size of the sample to use when computing the Silhouette
        Coefficient. If sample_size is None, no sampling is used.
    random_state : integer or numpy.RandomState, optional
        The generator used to initialize the centers. If an integer is
        given, it fixes the seed. Defaults to the global numpy random
        number generator.
    `**kwds` : optional keyword parameters
        Any further parameters are passed directly to the distance function.
        If using a scipy.spatial.distance metric, the parameters are still
        metric dependent. See the scipy docs for usage examples.
    Returns
    -------
    silhouette : float
        Mean Silhouette Coefficient for all samples.
    References
    ----------
    Peter J. Rousseeuw (1987). "Silhouettes: a Graphical Aid to the
        Interpretation and Validation of Cluster Analysis". Computational
        and Applied Mathematics 20: 53-65. doi:10.1016/0377-0427(87)90125-7.
    http://en.wikipedia.org/wiki/Silhouette_(clustering)
    """
    if sample_size is not None:
        random_state = check_random_state(random_state)
        indices = random_state.permutation(X.shape[0])[:sample_size]
        if metric == "precomputed":
            raise ValueError('Distance matrix cannot be precomputed')
        else:
            X, labels = X[indices], labels[indices]
    sil_samp = silhouette_samples_block(
        X, labels, metric=metric, n_jobs=n_jobs, **kwds)
    return np.mean(sil_samp), sil_samp


def silhouette_samples_block(X, labels, metric='euclidean', n_jobs=1, **kwds):
    """Compute the Silhouette Coefficient for each sample.
    The Silhoeutte Coefficient is a measure of how well samples are clustered
    with samples that are similar to themselves. Clustering models with a high
    Silhouette Coefficient are said to be dense, where samples in the same
    cluster are similar to each other, and well separated, where samples in
    different clusters are not very similar to each other.
    The Silhouette Coefficient is calculated using the mean intra-cluster
    distance (a) and the mean nearest-cluster distance (b) for each sample.
    The Silhouette Coefficient for a sample is ``(b - a) / max(a, b)``.
    This function returns the Silhoeutte Coefficient for each sample.
    The best value is 1 and the worst value is -1. Values near 0 indicate
    overlapping clusters.
    Parameters
    ----------
    X : array [n_samples_a, n_features]
        Feature array.
    labels : array, shape = [n_samples]
             label values for each sample
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string, it must be one of the options
        allowed by metrics.pairwise.pairwise_distances. If X is the distance
        array itself, use "precomputed" as the metric.
    `**kwds` : optional keyword parameters
        Any further parameters are passed directly to the distance function.
        If using a scipy.spatial.distance metric, the parameters are still
        metric dependent. See the scipy docs for usage examples.
    Returns
    -------
    silhouette : array, shape = [n_samples]
        Silhouette Coefficient for each samples.
    References
    ----------
    Peter J. Rousseeuw (1987). "Silhouettes: a Graphical Aid to the
        Interpretation and Validation of Cluster Analysis". Computational
        and Applied Mathematics 20: 53-65. doi:10.1016/0377-0427(87)90125-7.
    http://en.wikipedia.org/wiki/Silhouette_(clustering)
    """
    A = _intra_cluster_distances_block(X, labels, metric, n_jobs=n_jobs,
                                       **kwds)
    B = _nearest_cluster_distance_block(X, labels, metric, n_jobs=n_jobs,
                                        **kwds)
    sil_samples = (B - A) / np.maximum(A, B)
    # nan values are for clusters of size 1, and should be 0
    return np.nan_to_num(sil_samples)


def _intra_cluster_distances_block_(subX, metric, **kwds):
    distances = pairwise_distances(subX, metric=metric, **kwds)
    return distances.sum(axis=1) / (distances.shape[0] - 1)


def _intra_cluster_distances_block(X, labels, metric, n_jobs=1, **kwds):
    """Calculate the mean intra-cluster distance for sample i.
    Parameters
    ----------
    X : array [n_samples_a, n_features]
        Feature array.
    labels : array, shape = [n_samples]
        label values for each sample
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string, it must be one of the options
        allowed by metrics.pairwise.pairwise_distances. If X is the distance
        array itself, use "precomputed" as the metric.
    `**kwds` : optional keyword parameters
        Any further parameters are passed directly to the distance function.
        If using a scipy.spatial.distance metric, the parameters are still
        metric dependent. See the scipy docs for usage examples.
    Returns
    -------
    a : array [n_samples_a]
        Mean intra-cluster distance
    """
    intra_dist = np.zeros(labels.size, dtype=float)
    values = Parallel(n_jobs=n_jobs)(
            delayed(_intra_cluster_distances_block_)
                (X[np.where(labels == label)[0]], metric, **kwds)
                for label in np.unique(labels))
    for label, values_ in zip(np.unique(labels), values):
        intra_dist[np.where(labels == label)[0]] = values_
    return intra_dist


def _nearest_cluster_distance_block_(subX_a, subX_b, metric, **kwds):
    dist = pairwise_distances(subX_a, subX_b, metric=metric, **kwds)
    dist_a = dist.mean(axis=1)
    dist_b = dist.mean(axis=0)
    return dist_a, dist_b


def _nearest_cluster_distance_block(X, labels, metric, n_jobs=1, **kwds):
    """Calculate the mean nearest-cluster distance for sample i.
    Parameters
    ----------
    X : array [n_samples_a, n_features]
        Feature array.
    labels : array, shape = [n_samples]
        label values for each sample
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string, it must be one of the options
        allowed by metrics.pairwise.pairwise_distances. If X is the distance
        array itself, use "precomputed" as the metric.
    `**kwds` : optional keyword parameters
        Any further parameters are passed directly to the distance function.
        If using a scipy.spatial.distance metric, the parameters are still
        metric dependent. See the scipy docs for usage examples.
    X : array [n_samples_a, n_features]
        Feature array.
    Returns
    -------
    b : float
        Mean nearest-cluster distance for sample i
    """
    inter_dist = np.empty(labels.size, dtype=float)
    inter_dist.fill(np.inf)
    # Compute cluster distance between pairs of clusters
    unique_labels = np.unique(labels)

    values = Parallel(n_jobs=n_jobs)(
            delayed(_nearest_cluster_distance_block_)(
                X[np.where(labels == label_a)[0]],
                X[np.where(labels == label_b)[0]],
                metric, **kwds)
                for label_a, label_b in combinations(unique_labels, 2))

    for (label_a, label_b), (values_a, values_b) in \
            zip(combinations(unique_labels, 2), values):

            indices_a = np.where(labels == label_a)[0]
            inter_dist[indices_a] = np.minimum(values_a, inter_dist[indices_a])
            del indices_a
            indices_b = np.where(labels == label_b)[0]
            inter_dist[indices_b] = np.minimum(values_b, inter_dist[indices_b])
            del indices_b
    return inter_dist

In [14]:
metrics_df = pd.DataFrame(columns = ['dataset', 'class_', 'dim', 'trustworthiness', 'silhouette_score', 'silhouette_samples'])

### CIFAR10

##### load dataset

In [15]:
from tensorflow.keras.datasets import cifar10

# load dataset
(train_images, Y_train), (test_images, Y_test) = cifar10.load_data()
X_train = (train_images/255.).astype('float32')
X_test = (test_images/255.).astype('float32')
X_train = X_train.reshape((len(X_train), np.product(np.shape(X_train)[1:])))
X_test = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

# subset a validation set
n_valid = 10000
X_valid = X_train[-n_valid:]
Y_valid = Y_train[-n_valid:].flatten()
X_train = X_train[:-n_valid]
Y_train = Y_train[:-n_valid].flatten()
Y_test = Y_test.flatten()

print(len(X_train), len(X_valid), len(X_test))

40000 10000 10000


In [25]:
X_train_flat = X_train.reshape((len(X_train), 32*32*3))

#### load projections

In [16]:
dataset = 'cifar10'

In [17]:
classes = ['umap-learn', 'direct', 'network', 'autoencoder', 'PCA', 'TSNE']

In [18]:
projection_df = pd.DataFrame(columns = ['dataset', 'class_', 'train_z', 'train_label'])
for class_ in classes:
    print(output_dir / dataset / class_)
    z = np.load(output_dir / dataset / class_ / 'z.npy')
    projection_df.loc[len(projection_df)] = [dataset, class_, z, Y_train]
projection_df['dim'] = 2

/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/umap-learn
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/direct
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/network
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/autoencoder
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/PCA
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/TSNE


In [19]:
projection_df_64 = pd.DataFrame(columns = ['dataset', 'class_', 'train_z', 'train_label'])
for class_ in classes:
    print(output_dir / dataset / '64' / class_ / 'z.npy')
    try:
        z = np.load(output_dir / dataset / '64' / class_ / 'z.npy')
        projection_df_64.loc[len(projection_df_64)] = [dataset, class_, z, Y_train]
    except FileNotFoundError as e:
        print(e)
projection_df_64['dim'] = 64

/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/umap-learn/z.npy
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/direct/z.npy
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/network/z.npy
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/autoencoder/z.npy
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/PCA/z.npy
/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/TSNE/z.npy
[Errno 2] No such file or directory: '/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/TSNE/z.npy'


In [20]:
projection_df_64

,dataset,class_,train_z,train_label,dim
0,cifar10,umap-learn,"[[7.1060157, 9.195796, 7.6382084, 7.2188716, 5...","[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, ...",64
1,cifar10,direct,"[[1.0066352, -0.98004335, -0.34852365, 0.21314...","[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, ...",64
2,cifar10,network,"[[-0.05073797, 0.15603174, -0.20267391, 0.0221...","[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, ...",64
3,cifar10,autoencoder,"[[0.09604484, -0.09921512, -0.10305964, -0.012...","[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, ...",64
4,cifar10,PCA,"[[-6.3943524, 2.7541623, 1.4807999, -1.903707,...","[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, ...",64


##### compute silhouette score and trustworthiness

In [28]:
for idx, row in tqdm(pd.concat([projection_df, projection_df_64]).iterrows(), total = len(projection_df)+ len(projection_df_64)):
    #ss, sil_samp = silhouette_score_block(row.train_z, row.train_label, n_jobs = -1)
    tw = trustworthiness(X_train_flat[:10000], row.train_z[:10000])
    print(tw)
    metrics_df.loc[len(metrics_df)] = [dataset, row.class_, row.dim, tw, np.nan, np.nan]

0.8309924699759808
0.8451649739791833
0.8186705584467574
0.8273451261008807
0.8201575820656526
0.9215858426741393
0.9209458206565252
0.9104933847077662
0.9139511709367494
0.9199400760608487
0.9995521697357886


In [29]:
metrics_df

,dataset,class_,dim,trustworthiness,silhouette_score,silhouette_samples
0,cifar10,umap-learn,2,0.830992,NaN,NaN
1,cifar10,umap-learn,2,0.830992,NaN,NaN
2,cifar10,direct,2,0.845165,NaN,NaN
3,cifar10,network,2,0.818671,NaN,NaN
4,cifar10,autoencoder,2,0.827345,NaN,NaN
5,cifar10,PCA,2,0.820158,NaN,NaN
6,cifar10,TSNE,2,0.921586,NaN,NaN
7,cifar10,umap-learn,64,0.920946,NaN,NaN
8,cifar10,direct,64,0.910493,NaN,NaN
9,cifar10,network,64,0.913951,NaN,NaN
